# ActiveRecord

Zanim przystąpimy do pracy, musimy skonfigurować bazę danych. Ponieważ pracujemy bez użycia Railsów, konieczne jest
ręczne zestawienie połączenia oraz stworzenie odpowiednich tabel w bazie. Zadania te realizowane są przez
skrypt `db_setup.rb`. Aby go uruchomić wpisujemy
```ruby
$:.unshift "."
require 'db_setup'
```

In [1]:
$:.unshift "."
require 'db_setup'

==  CreateAuthorsMigration: migrating =========================================
-- create_table(:authors)
   -> 0.0873s
==  CreateAuthorsMigration: migrated (0.0921s) ================================

==  CreateBooksMigration: migrating ===========================================
-- create_table(:books)
   -> 0.0342s
==  CreateBooksMigration: migrated (0.0500s) ==================================

==  CreateGenresMigration: migrating ==========================================
-- create_table(:genres)
   -> 0.0295s
==  CreateGenresMigration: migrated (0.0343s) =================================

==  CreateBooksGenresMigration: migrating =====================================
-- create_table(:books_genres, {:id=>false})
   -> 0.0259s
-- add_index(:books_genres, [:book_id, :genre_id], {:unique=>true})
   -> 0.2434s
==  CreateBooksGenresMigration: migrated (0.2965s) ============================

==  AddIndicesMigration: migrating ============================================
==  AddIndicesMigr

true

W dalszych zadaniach będzie wykonywać polecenia korzystając z następujących klas, zmapowanych na odpowiadające im 
table w bazie danych:
```ruby
class Author < ActiveRecord::Base
  # name      (string)
  # surname   (string)
  # born      (datetime) 
  # died      (datetime)
  # image_url (string)
  
  has_many :books
end

class Book < ActiveRecord::Base
  # title     (string)
  # language  (string)
  # author    (Author)
  # published (integer)

  belongs_to :author
  has_and_belongs_to_many :genres
end

class Genre < ActiveRecord::Base
  # name  (string)
  
  has_and_belongs_to_many :books
end
```

## CRUD

Cztery podstawowe operacje, które wykonujemy na danych to
* tworzenie - **C**reate
* odczytywanie - **R**ead
* modyfikowanie - **U**pdate
* usuwanie - **D**elete

W skrócie oznaczane są one za pomocą akronimu CRUD.

### Create

W ActiveRecord (w skrócie AR) korzystamy z obiektowego interfejsu. 
Tworzenie danych wygląda następująco:
```ruby
author = Author.new(name: "Adam", surname: "Mickiewicz")
author.save
```

In [2]:
author = Author.new(name: "Adam", surname: "Mickiewicz")
author.save


true

Innymi słowy tworzymy nowy obiekt Rubiego i wywołujemy na nim metodę `save`. To że faktycznie został on dodany do bazy danych 
możemy zweryfikować wyszukując pierwszy obiekt w bazie:
```ruby
author = Author.first
puts author.name
puts author.surname
```

In [3]:
author = Author.first
puts author.name
puts author.surname


Adam
Mickiewicz


### Zadanie 1

Dodaj do bazy 3 autorów:
* Juliusz Słowacki
* Henryk Sienkiewicz
* Eliza Orzeszkowa

In [4]:
author = Author.new(name: "Juliusz", surname: "Słowacki")
author.save
author = Author.new(name: "Henryk", surname: "Sienkiewicz")
author.save
author = Author.new(name: "Eliza", surname: "Orzeszkowa")
author.save


true

### Read

Odczytywanie danych z bazy można realizować na wiele sposobów. Najprostszy sposób, to wyszukiwanie ich z wykorzystaniem 
klucza główego - `id`. Służy do tego metoda `find`:
```ruby
author = Author.find(1)
puts author.surname
author = Author.find(2)
puts author.surname
```

In [5]:
author = Author.find(1)
puts author.surname
author = Author.find(2)
puts author.surname


Mickiewicz
Słowacki


Wykorzystanie tej metody może jedak skutkować wyjątkiem, jeśli w bazie nie ma wiersza z danym kluczem:
```ruby
author = Author.find(10)
```

In [6]:
author = Author.find(10)

ActiveRecord::RecordNotFound: Couldn't find Author with 'id'=10

Możemy zabezpieczyć się przed tą sytuacją, korzystają z innego wywołania `find_by_id`
```ruby
author = Author.find_by_id(10)
p author
```

In [7]:
author = Author.find_by_id(10)
p author


nil


### Zadanie 2

Próba odczytania pól takiego obiektu, również skończy się wyjątkiem. Co należy zrobić, żeby wypisać imię i nazwisko autora
wyłącznie wtedy gdy autor istnieje w bazie? Zaimplementuj metodę `print_author`, która radzi sobie z tym problemem.

In [10]:
def print_author(id)
  author = Author.find_by_id(id)
  if Author.find_by_id(id) == nil
    p "Nie ma takiego id."
  else
    puts author.name + " " + author.surname
  end

  
end

# te linijki mają pozostać niezmienione
print_author(1)
print_author(10)

Adam Mickiewicz
"Nie ma takiego id."


"Nie ma takiego id."

### Update

Modyfikowanie danych realizowane może być na kilka sposobów. W pierwszej kolejności możemy zmodyfikować atrybut obiektu i 
następnie zapisać go do bazy
```ruby
author = Author.find(1)
puts author.surname
author.surname = "Mickiewiczowski"
author.save

other_author = Author.find(1)
puts other_author.surname
```

In [12]:
author = Author.find(1)
puts author.surname
author.surname = "Mickiewiczowski"
author.save

other_author = Author.find(1)
puts other_author.surname

author = Author.find(1)
author.update_attributes(name: "Wojciech")
other_author = Author.find(1)
puts other_author.name



Mickiewiczowski
Mickiewiczowski
Wojciech


Można również skorzystać z metody `update_attributes`, która działa podobnie jak konstruktor, ale dane są od razu
modyfikowane w bazie
```ruby
author = Author.find(1)
author.update_attributes(name: "Wojciech")
other_author = Author.find(1)
puts other_author.name
```

### Zadanie 3

Zmodyfikuj wszystkich autorów, tak by ich daty urodzenia i śmierci były poprawne. Popraw również imię i nazwisko Adama Mickiewicza.
Aby wprowadzić datę skorzystaj z metody `Date.parse`.

In [23]:
author = Author.find(5)
author.update_attributes(name: "Adam", surname: "Mickiewicz", born: Date.parse("1789-12-24"), died: Date.parse("1855-11-26"))
other_author = Author.find(5)
puts other_author.name
puts other_author.surname
puts other_author.born
puts other_author.died
author = Author.find(2)
author.update_attributes(name: "Juliusz", surname: "Słowacki", born: Date.parse("1809-09-04"), died: Date.parse("1849-04-03"))
other_author = Author.find(2)
puts other_author.name
puts other_author.surname
puts other_author.born
puts other_author.died
author = Author.find(3)
author.update_attributes(name: "Henryk", surname: "Sienkiewicz", born: Date.parse("1846-05-05"), died: Date.parse("1916-11-05"))
other_author = Author.find(3)
puts other_author.name
puts other_author.surname
puts other_author.born
puts other_author.died
author = Author.find(4)
author.update_attributes(name: "Eliza", surname: "Orzeszkowa", born: Date.parse("1841-06-06"), died: Date.parse("1910-05-23"))
other_author = Author.find(4)
puts other_author.name
puts other_author.surname
puts other_author.born
puts other_author.died

Adam
Mickiewicz
1789-12-24
1855-11-26
Juliusz
Słowacki
1809-09-04
1849-04-03
Henryk
Sienkiewicz
1846-05-05
1916-11-05
Eliza
Orzeszkowa
1841-06-06
1910-05-23


### Delete

Usuwanie danych realizowane jest za pomocą wywołania `destroy`:
```ruby
author = Author.find(1)
author.destroy
```

In [15]:
author = Author.find(1)
author.destroy


#<Author id: 1, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>

### Zadanie 4

Ponieważ właśnie usunąłeś/ęłaś Adama Mickiewicza, ponownie utwórz odpowiadający mu rekord.

In [16]:
author = Author.new(name: "Adam", surname: "Mickiewicz")
author.save

true

## Język zapytań

## `find`, `first`, `last`, `all`

Metoda `find` pozwala nie tylko pobierać pojedynczy obiekt z bazy, ale również kilka obiektów na raz:
```ruby
authors = Author.find(2,3,4)
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end
```

In [17]:
authors = Author.find(2,3,4)
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end


Juliusz Słowacki
Henryk Sienkiewicz
Eliza Orzeszkowa


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: nil, died: nil, image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: nil, died: nil, image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: nil, died: nil, image_url: nil>]

Metody `first` oraz `last` zwracają odpowiednio *pierwszy* i *ostatni* rekord w bazie. W domyślnej konfiguracji kolejność
ta będzie odpowiadała czasowi ich utworzenia.
```ruby
puts Author.first.surname
puts Author.last.surname
```

In [18]:
puts Author.first.surname
puts Author.last.surname


Słowacki
Mickiewicz


Natomiast metoda `all` zwraca kolekcję obejmującą wszystkie rekordy w bazie danych:
```ruby
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end
```

In [19]:
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname}"
end


Juliusz Słowacki
Henryk Sienkiewicz
Eliza Orzeszkowa
Adam Mickiewicz


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: nil, died: nil, image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: nil, died: nil, image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: nil, died: nil, image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: nil, died: nil, image_url: nil>]

### Zadanie 5

Wypisz wszystkich autorów znajdujących się w bazie wraz z ich datami urodzenia i śmierci. Postaraj się sformatować daty,
tak by obejmowały tylko dzień, miesiąc i rok - w tej kolejności. Służy do tego metoda `strftime`.

In [24]:
authors = Author.all
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")} #{author.died.strftime("%d-%m-%Y")}" 
end

Juliusz Słowacki 04-09-1809 03-04-1849
Henryk Sienkiewicz 05-05-1846 05-11-1916
Eliza Orzeszkowa 06-06-1841 23-05-1910
Adam Mickiewicz 24-12-1789 26-11-1855


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

### `find_by`

AR definiuje również metody pozwalające na wyszukiwanie rekordów na podstawie wartości atrybutów. Najprostsza z nich to `find_by`. Zwraca rekord, które posiada wartość określoną w zapytaniu:
```ruby
author = Author.find_by_name("Adam")
puts author.surname
```

In [25]:
author = Author.find_by_name("Adam")
puts author.surname


Mickiewicz


### `where`

Metoda `where` odpowiada klauzuli `where` z języka SQL. Podstawowa różnica polega na tym, że wartości poszczególnych pól określamy w postaci par klucz-wartość. Jeśli chcemy uzyskać pojedynczy wynik dodajemy metodę `first` lub `last`:
```ruby
author = Author.where(name: "Eliza").first
puts author.surname
```

In [26]:
author = Author.where(name: "Eliza").first
puts author.surname


Orzeszkowa


Metoda ta ma jednak znacznie większe możliwości - można np. podawać zakresy wartości, jako zakresy Rubiego:
```ruby
authors = Author.where(born: (Date.parse("1780-1-1")..Date.parse("1800-12-31"))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end
```

In [32]:
authors = Author.where(born: (Date.parse("1780-1-1")..Date.parse("1800-12-31")))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born.strftime("%d-%m-%Y")}"
end



Adam Mickiewicz 24-12-1789


[#<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

Metodę `where` można wywoływać wielokrotnie. Wtedy wyniki są łączone za pomocą operatora koniunkcji. Jeśli chcemy
użyć innego operatora (np. `OR` lub `LIKE`), konieczne jest użycie nieco innej składni:
```ruby
authors = Author.where("name LIKE 'A%'")
authors.each do |author|
  puts author.surname
end
```  

In [33]:
authors = Author.where("name LIKE 'A%'")
authors.each do |author|
  puts author.surname
end


Mickiewicz


[#<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

Jeśli dane w napisie przekazanym do metody `where` pochodzą od użytkownika aplikacji, to narażamy się na atak SQL-injection.
Aby go unikąć, wartość podaną przez użytkownika przekazujemy jako osobny argument, np.
```ruby
name = "Adam"
authors = Author.where("name = ?",name)
authors.each do |author|
  puts author.surname
end
```

In [34]:
name = "Adam"
authors = Author.where("name = ?",name)
authors.each do |author|
  puts author.surname
end


Mickiewicz


[#<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

W tej sytuacji AR sam zadba o odpowiednią konwersję znaków "niebezpiecznych".

### Zadanie 6

Znajdź i wypisz wszystkich autorów, którzy zmarli między rokiem 1800 a 1900.

In [35]:
authors = Author.where(died: (Date.parse("1800-1-1")..Date.parse("1900-1-1")))
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.died.strftime("%d-%m-%Y")}"
end

Juliusz Słowacki 03-04-1849
Adam Mickiewicz 26-11-1855


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

### `order`

Do określania kolejności wyników służy metoda `order`. Działa ona analogicznie do klauzuli `ORDER` w języku SQL.
```ruby
authors = Author.order(:born)
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [36]:
authors = Author.order(:born)
authors.each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end


Adam Mickiewicz 1789-12-24
Juliusz Słowacki 1809-09-04
Eliza Orzeszkowa 1841-06-06
Henryk Sienkiewicz 1846-05-05


[#<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>]

Metoda ta często jest łączona z wywołaniami `first` i `last`
```ruby
author = Author.order(:born).last
puts "#{author.name} #{author.surname} #{author.born}"
```

### Zadanie 7

Znajdź autora, który zmarł jako ostatni.

In [37]:
author = Author.order(:died).last
puts "#{author.name} #{author.surname} #{author.died}"


Henryk Sienkiewicz 1916-11-05


### `limit` i `offset`

Metody `limit` i `offset` działają analogicznie jak ich odpowiedniki w SQL:
```ruby
Author.limit(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [38]:
Author.limit(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end


Juliusz Słowacki 1809-09-04
Henryk Sienkiewicz 1846-05-05


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 3, name: "Henryk", surname: "Sienkiewicz", born: "1846-05-05", died: "1916-11-05", image_url: nil>]

```ruby
Author.offset(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end
```

In [39]:
Author.offset(2).each do |author|
  puts "#{author.name} #{author.surname} #{author.born}"
end



Eliza Orzeszkowa 1841-06-06
Adam Mickiewicz 1789-12-24


[#<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>]

Warto jednak pamiętać, żeby stosując je określić pożądek rekordów.

### Zadanie 8

Znajdź i wypisz 3 autorów, którzy zmarli jako pierwsi.

In [40]:
Author.order(:died).limit(3).each do |author|
  puts "#{author.name} #{author.surname} #{author.died}"
end

Juliusz Słowacki 1849-04-03
Adam Mickiewicz 1855-11-26
Eliza Orzeszkowa 1910-05-23


[#<Author id: 2, name: "Juliusz", surname: "Słowacki", born: "1809-09-04", died: "1849-04-03", image_url: nil>, #<Author id: 5, name: "Adam", surname: "Mickiewicz", born: "1789-12-24", died: "1855-11-26", image_url: nil>, #<Author id: 4, name: "Eliza", surname: "Orzeszkowa", born: "1841-06-06", died: "1910-05-23", image_url: nil>]